# Disambiguierung der Web Of Science Institutionen

- Dieses Notebook beschäftigt sich mit der Erkennung und Disambiguierung der Institutionen aus Web of Science.
- Die Institutionen sind ein wichtiger Bestandteil für die Erstellung der Lebensläufe von Wissenschaftlern.
- Es wird zunächst ein Auszug aus den Rohdaten gezeigt um die Notwendigkeit einer Disambiguation aufzuzeigen. 
- Anschließend wird stark vereinfacht gezeigt, wie mit Hilfe von Wikidata und der Cosinus Similarity eine Disambiguation durchgeführt werden kann.
- Zum Abschluss wird der fertige Institutionen-Matcher in seiner Anwendung gezeigt (kein Code).

In [1]:
import pandas as pd # zur Darstellung von Tabellen
import re # reguläre Expressionen für z.B. Cleaning von Text
import src # enthält den relativen Ordnerpfad des Notebook
import sqlite3 #zum Laden von Datenbanken
from collections import Counter # Zähler für Wörter
import math # mathematische Werkzeuge

# zum Abrufen von Daten aus dem Internet, hier Wikidata
import sys, requests
from SPARQLWrapper import SPARQLWrapper, JSON #!pip install SPARQLWrapper

## Rohdaten (Auszug aus WoS)

Die Web of Science Daten enthalten:

- PK_INSTITUTIONS 
    - eine einzigarte ID
- ORGANIZATION1 
    - erster Teil der Institution, meistens die höchste Instanz der Organisation
- ORGANIZATION2
    - zweiter Teil der Institution, meistens die zweithöchste Instanz der Organisation
- ORGANIZATION3
    - dritter Teil der Institution, meistens die dritthöchste Instanz der Organisation
- ORGANIZATION4
    - vierter Teil der Institution, meistens die vierthöchste Instanz der Organisation
- INSTITUTION_FULL
    - ORGANIZATION1 + ORGANIZATION2 + ORGANIZATION3 + ORGANIZATION4
- POSTALCODE
    - meistens die Postleitzahl
- CITY 
    - meistens die Stadt
- ADDRESS_FULL
    - INSTITUTION_FULL + CITY + POSTALCODE + COUNTRYCODE
- COUNTRYCODE
    - Land der Institution

In [2]:
db = sqlite3.connect(src.PATH / "data/sample.db")
cur = db.cursor()
df = pd.DataFrame(cur.execute("""select * from wos_institutions""").fetchall(), columns=[c[0] for c in cur.description])
del df['index']
df.head()

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
0,24,Kings Coll London,None,None,None,Kings Coll London,None,London,"Kings Coll London, London, England",GBR
1,93,Kings Coll London,Dept Psychosis Studies,Inst Psychiat,None,"Kings Coll London, Dept Psychosis Studies, Inst Psychiat",None,London,"Kings Coll London, Inst Psychiat, Dept Psychosis Studies, London, England",GBR
2,159,Univ Konstanz,Dept Psychol,None,None,"Univ Konstanz, Dept Psychol",None,Constance,"Univ Konstanz, Dept Psychol, Constance, Germany",DEU
3,530,Deutsch Herzzentrum Munich,None,None,None,Deutsch Herzzentrum Munich,None,Munich,"Deutsch Herzzentrum Munich, Munich, Germany",DEU
4,1047,Friedrich Loeffler Inst,None,None,None,Friedrich Loeffler Inst,None,Insel Riems,"Friedrich Loeffler Inst, Insel Riems, Germany",DEU


In [3]:
# Meistens unterscheiden sich die Institutionen nur minimal von einander (vor allem in ihren Unter-Organisationen und PLZ):
kings_college = df[df['ORGANIZATION1'].str.contains('Kings Coll') & ~df['ORGANIZATION1'].isnull()]
print( 'Varianten nur für das Kings College in London (Teil der Universät London):', len(kings_college) )
kings_college.head()

Varianten nur für das Kings College in London (Teil der Universät London): 105


,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
0,24,Kings Coll London,None,None,None,Kings Coll London,None,London,"Kings Coll London, London, England",GBR
1,93,Kings Coll London,Dept Psychosis Studies,Inst Psychiat,None,"Kings Coll London, Dept Psychosis Studies, Inst Psychiat",None,London,"Kings Coll London, Inst Psychiat, Dept Psychosis Studies, London, England",GBR
72,6161568,Kings Coll London,Eating Disorders Res Unit,Inst Psychiat,Div Psychol Med,"Kings Coll London, Eating Disorders Res Unit, Inst Psychiat, Div Psychol Med",SE5 8AF,London,"Kings Coll London, Div Psychol Med, Eating Disorders Res Unit, Inst Psychiat, London SE5 8AF, England",GBR
439,2357,Kings Coll London,Dept Phys,None,None,"Kings Coll London, Dept Phys",None,London,"Kings Coll London, Dept Phys, London, England",GBR
599,12192284,Kings Coll London,Div Psychol Med & Psychiat,Sect Eating Disorders,Inst Psychiat,"Kings Coll London, Div Psychol Med & Psychiat, Sect Eating Disorders, Inst Psychiat",WC2R 2LS,London,"Kings Coll London, Inst Psychiat, Div Psychol Med & Psychiat, Sect Eating Disorders, London WC2R 2LS, England",GBR


In [4]:
# Es kann jedoch vorkommen, dass nicht immer die höchste Instanz angegeben ist:
kings_college.drop_duplicates('ORGANIZATION1')

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
0,24,Kings Coll London,None,None,None,Kings Coll London,None,London,"Kings Coll London, London, England",GBR
21747,33251576,Kings Coll Hosp NHS Fdn Trust,Inst Liver Studies,None,None,"Kings Coll Hosp NHS Fdn Trust, Inst Liver Studies",None,London,"Kings Coll Hosp NHS Fdn Trust, Inst Liver Studies, London, England",GBR
23121,28931411,Univ London Kings Coll,None,None,None,Univ London Kings Coll,WC2R 2LS,London,"Univ London Kings Coll, London WC2R 2LS, England",GBR


In [5]:
# Es gibt verschieden Schreibweisen wie Abkürzungen (auch in verschiedenen Spachen):
df[df['ORGANIZATION1'].str.contains('KCL') & ~df['ORGANIZATION1'].isnull()]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
14371,33875386,KCL,Inst Psychiat Psychol & Neurosci,Psychol Med,None,"KCL, Inst Psychiat Psychol & Neurosci, Psychol Med",None,London,"KCL, Psychol Med, Inst Psychiat Psychol & Neurosci, London, England",GBR


In [6]:
# Andere Universitäten können Unterinstanzen mit ähnlichen oder sogar identischen Namen haben:
df[df['ORGANIZATION4'].str.contains('Kings Coll') & ~df['ORGANIZATION4'].isnull()]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
12104,1099229,Univ London,Inst Psychiat Maudsley,Eating Disorders Unit,Kings Coll,"Univ London, Inst Psychiat Maudsley, Eating Disorders Unit, Kings Coll",None,London,"Univ London, Kings Coll, Inst Psychiat Maudsley, Eating Disorders Unit, London, England",GBR
17274,5708507,Univ Aberdeen,Sch Psychol,Dept Psychol,Kings Coll,"Univ Aberdeen, Sch Psychol, Dept Psychol, Kings Coll",AB24 3FX,Aberdeen,"Univ Aberdeen, Kings Coll, Sch Psychol, Dept Psychol, Aberdeen AB24 3FX, Scotland",GBR


In [7]:
# Dies kann innerhalb von Web of Science zu eigenartigen Datenpunkten führen:
df[df['ORGANIZATION3'].str.contains('Kings Coll') & ~df['ORGANIZATION3'].isnull()]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
17308,7151195,Univ Aberdeen,Sch Psychol,Univ London Kings Coll,None,"Univ Aberdeen, Sch Psychol, Univ London Kings Coll",None,Aberdeen,"Univ Aberdeen, Univ London Kings Coll, Sch Psychol, Aberdeen, Scotland",GBR


In [8]:
# Es gibt auch Datenpunkte mit sehr wenig Informationen:
df[df['PK_INSTITUTIONS']==2782303]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
6996,2782303,None,None,None,None,None,E11 1PD,London,"6 Eagle Court, London E11 1PD, England",GBR


In [9]:
# Es gibt keine Garantie für eine richtige Schreibweise:
df[df['PK_INSTITUTIONS']==22795164]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
28958,22795164,Univ Colognem,Dept Psychol,None,None,"Univ Colognem, Dept Psychol",D-50969,Cologne,"Univ Colognem, Dept Psychol, Richard Strauss Str 2, D-50969 Cologne, Germany",DEU


In [10]:
# Es gibt auch Organisationen, die nicht unserem Beuteschema entsprechen:
df[df['PK_INSTITUTIONS']==3992807]

,PK_INSTITUTIONS,ORGANIZATION1,ORGANIZATION2,ORGANIZATION3,ORGANIZATION4,INSTITUTION_FULL,POSTALCODE,CITY,ADDRESS_FULL,COUNTRYCODE
14429,3992807,qutools GmbH,None,None,None,qutools GmbH,D-80539,Munich,"qutools GmbH, D-80539 Munich, Germany",DEU


### Problematik:

- Welche Datenpunkte sind Universitäten, Hochschulen, etc. und welche nicht?
- Wie geht man mit unterschiedlichen Schreibweisen und -fehlern um?
- Welche Datenpunkte bilden die selbe Institution ab?


### Lösungsansatz:
- Datenbank mit saubern Institutionen aufbauen
- Über die Cosinus-Similarity ähnliche Schreibweisen matchen
- Über die Berechnung eines Scores (Gewichtung von Name, Stadt, etc.) disambiguieren

# Metadata (Wikidata)

Es wird die Wikidata API genutzt um alles Universitäten und ähnliche Institutionen zu erheben.

### Ausführliche Erläuterung in der nächsten Sitzung

WikiData-Entity: https://www.wikidata.org/wiki/Q245247

WikiData-Query:
https://query.wikidata.org/#%23King%27s%20College%20Q245247%0ASELECT%20%3Flabel%20%3Faka%20%3Flocation%20%3FlocationLabel%20%3Fcountry%20%3FcountryLabel%20%3Fioc%20%3Fparent%20%3FparentLabel%0AWHERE%20%0A%7B%0A%20%20VALUES%20%3Fitem%20%7B%20wd%3AQ245247%20%7D%0A%20%20OPTIONAL%7B%3Fitem%20rdfs%3Alabel%20%3Flabel.%20FILTER%20%28langMatches%28%20lang%28%3Flabel%29%2C%20%22EN%22%20%29%20%29%7D%0A%20%20%23OPTIONAL%7B%3Fitem%20skos%3AaltLabel%20%3Faka.%7D%20%23%20also%20known%20as%0A%20%20OPTIONAL%7B%3Fitem%20wdt%3AP276%20%3Flocation.%7D%20%23%20location%0A%20%20OPTIONAL%7B%3Fitem%20wdt%3AP17%20%3Fcountry.%7D%20%23%20country%0A%20%20OPTIONAL%7B%3Fcountry%20wdt%3AP984%20%3Fioc.%7D%20%23%20country%0A%20%20OPTIONAL%7B%3Fitem%20wdt%3AP361%20%3Fparent.%7D%20%23%20parent%20of%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%7D

In [11]:
# Abrufen der Daten einer Institution über Wikidata

# https://rdflib.github.io/sparqlwrapper/

endpoint_url = "https://query.wikidata.org/sparql"

query = """#King's College Q245247
SELECT ?label ?aka ?location ?locationLabel ?country ?countryLabel ?ioc ?parent ?parentLabel
WHERE 
{
  VALUES ?item { wd:Q245247 }
  OPTIONAL{?item rdfs:label ?label. FILTER (langMatches( lang(?label), "EN" ) )}
  OPTIONAL{?item skos:altLabel ?aka.} # also known as
  OPTIONAL{?item wdt:P276 ?location.} # location
  OPTIONAL{?item wdt:P17 ?country.} # country
  OPTIONAL{?country wdt:P984 ?ioc.} # country
  OPTIONAL{?item wdt:P361 ?parent.} # parent of
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

# entferne leere Ergebnisse
def try2unpack(x):
    try:
        x=x['value']
    except (KeyError, TypeError):
        pass
    
    try:
        if 'http://www.wikidata.org/entity/Q' in x or 'http://www.wikidata.org/entity/P' in x:
            return x.rsplit('/',1)[1]
        else:
            return x
    except TypeError:
        return x

# transformiere in brauchbare Tabelle
def json2pandas(data):
    return pd.DataFrame(data['results']['bindings'], columns=data['head']['vars']).applymap(lambda x: try2unpack(x))

json2pandas(get_results(endpoint_url, query))

,label,aka,location,locationLabel,country,countryLabel,ioc,parent,parentLabel
0,King's College London,Kingov koledž,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
1,King's College London,Kingov koledž,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
2,King's College London,Kings koledž,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
3,King's College London,Kings koledž,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
4,King's College London,Kings koledž,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
...,...,...,...,...,...,...,...,...,...
133,King's College London,Кингс-колледж,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
134,King's College London,King's College London,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
135,King's College London,King's College London,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London
136,King's College London,King's College London,Q84,London,Q145,United Kingdom,GBR,Q170027,University of London


#### Wie man sieht, bekommen wir alle nötigen Informationen aus Wikidata.
#### Neben Namensvariationen, Land, Stadt etc. bietet Wikidata auch noch weitere Informationen wie z.B. die höhere Instanz (parent) und vieles mehr.

In [12]:
# Abrufen der Daten aller Institutionen über Wikidata

# get all items in specific classes (ordered by organizational size / hierarchy)
get_classes={"Q1075106":"1102_UNI_system", # e.g: University of Massachusetts
             "Q3918":"1101_UNI", # e.g. University of Massachusetts Amherst
             "Q21028957":"1100_UNI_hs", # Hochschule e.g: Bucerius Law School
             "Q178706" :"0009_inst", # all kind of institute, e.g.: Faculty of Law of the Trnava University in Trnava
            }

all_Qs = pd.DataFrame()
for Q, label in get_classes.items():
    print(label)
    query=f"""SELECT DISTINCT ?item ?instance WHERE {{?item wdt:P31/wdt:P279* wd:{Q}; wdt:P31 ?instance.}}"""
    data = json2pandas(get_results(endpoint_url, query))
    data['label']=label
    all_Qs=all_Qs.append(data)

# keep the higher class: uni_system > uni > hochschule
all_Qs=all_Qs.sort_values('label', ascending=False)

all_Qs = all_Qs.drop_duplicates("item", keep='first')
print(len(all_Qs))

all_Qs.head()

1102_UNI_system
1101_UNI
1100_UNI_hs
0009_inst
444184


,item,instance,label
0,Q217439,Q3918,1102_UNI_system
28,Q2002043,Q1075106,1102_UNI_system
30,Q623581,Q45400320,1102_UNI_system
31,Q2140391,Q1075106,1102_UNI_system
32,Q2331177,Q1075106,1102_UNI_system


# Beispiel für eine Disambigueriung deutscher Institutionen

### Für das Beispiel nutzen wir zur Anschauung nur die ORGANISATION1 und CITY und beschränken uns auf deutsch und englische Schreibweisen.

In [13]:
# Wir Filtern die deutschen Institutionen aus dem Web of Science Sample
wos_sample = df[df['COUNTRYCODE']=='DEU'][['PK_INSTITUTIONS','ORGANIZATION1','CITY']]
wos_sample.head()

,PK_INSTITUTIONS,ORGANIZATION1,CITY
2,159,Univ Konstanz,Constance
3,530,Deutsch Herzzentrum Munich,Munich
4,1047,Friedrich Loeffler Inst,Insel Riems
5,131559,Tech Univ Munich,Am Coulombwall
6,131797,Univ Cologne,Cologne


https://query.wikidata.org/#SELECT%20DISTINCT%20%3Flabel_de%20%3Flabel_en%20%3Fitem%20%3Finstance%20%3Flocation1Label%20%3Flocation2Label%20%3Flocation3Label%20%3Fparent%20WHERE%20%7B%3Fitem%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ3918%3B%20wdt%3AP17%20wd%3AQ183%3B%20wdt%3AP31%20%3Finstance.%0A%20%20%20%20%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20rdfs%3Alabel%20%3Flabel_de.%20FILTER%20%28langMatches%28%20lang%28%3Flabel_de%29%2C%20%22de%22%20%29%20%29%7D%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20rdfs%3Alabel%20%3Flabel_en.%20FILTER%20%28langMatches%28%20lang%28%3Flabel_en%29%2C%20%22en%22%20%29%20%29%7D%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20wdt%3AP131%20%3Flocation1.%7D%20%23%20location1%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20wdt%3AP276%20%3Flocation2.%7D%20%23%20location2%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20wdt%3AP159%20%3Flocation3.%7D%20%23%20location3%0A%20%20%20%20%20%20OPTIONAL%7B%3Fitem%20wdt%3AP361%20%3Fparent.%7D%20%23%20parent%20of%0A%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cde%22.%20%7D%0A%20%20%20%20%7D%0A

In [14]:
# Wir holen uns mit dieser Query alle deutschen (Q183) Universitäten (Q3918)
query="""
SELECT DISTINCT ?label_de ?label_en ?item ?instance ?location1Label ?location2Label ?location3Label ?parent WHERE {?item wdt:P31/wdt:P279* wd:Q3918; wdt:P17 wd:Q183; wdt:P31 ?instance.
    
      OPTIONAL{?item rdfs:label ?label_de. FILTER (langMatches( lang(?label_de), "de" ) )}
      OPTIONAL{?item rdfs:label ?label_en. FILTER (langMatches( lang(?label_en), "en" ) )}
      OPTIONAL{?item wdt:P131 ?location1.} # location1
      OPTIONAL{?item wdt:P276 ?location2.} # location2
      OPTIONAL{?item wdt:P159 ?location3.} # location3
      OPTIONAL{?item wdt:P361 ?parent.} # parent of
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de". }
    }
""".strip()

# location1 -> oftmals Bezirke
# location2 -> adminstrative Bezikre
# location3 -> Hauptsitz

data = json2pandas(get_results(endpoint_url, query))
data=data.drop_duplicates('item')
data.head()

,label_de,label_en,item,instance,location1Label,location2Label,location3Label,parent
0,Friedrich-Alexander-Universität Erlangen-Nürnberg,University of Erlangen-Nuremberg,Q40025,Q62078547,Nürnberg,NaN,Erlangen,NaN
34,Martin-Luther-Universität Halle-Wittenberg,University of Halle-Wittenberg,Q32120,Q875538,Halle (Saale),NaN,NaN,NaN
38,Technische Universität Berlin,Technical University of Berlin,Q51985,Q1371037,Charlottenburg-Wilmersdorf,NaN,Hauptgebäude der Technischen Universität Berlin,NaN
39,Universität zu Köln,University of Cologne,Q54096,Q1767829,Köln,NaN,Köln,NaN
49,Goethe-Universität Frankfurt am Main,Goethe University Frankfurt,Q50662,Q641347,Frankfurt am Main,NaN,Jügelhaus,NaN


### Die Tabelle muss nun so umgeformt werden, dass wir nur noch ein Label und eine Location haben. Dazu werden normalerweise alle möglichen Kombinationen aus labels und locations generiert. Um die Datenmenge klein zu halten, behalten wir nur eine location und kombinieren dann mit den Labels.

In [15]:
# Behalte nur ein Label, dass am beste nicht None ist
def one_location(loc1, loc2, loc3):
    if loc1:
        return loc1
    if loc2:
        return loc2
    if loc3:
        return loc3
    return None

data['location'] = data[['location1Label','location2Label','location3Label']].apply(lambda x: one_location(x[0],x[1],x[2]), axis=1)
data = data[['label_de','label_en', 'item', 'location']]
data.head()

,label_de,label_en,item,location
0,Friedrich-Alexander-Universität Erlangen-Nürnberg,University of Erlangen-Nuremberg,Q40025,Nürnberg
34,Martin-Luther-Universität Halle-Wittenberg,University of Halle-Wittenberg,Q32120,Halle (Saale)
38,Technische Universität Berlin,Technical University of Berlin,Q51985,Charlottenburg-Wilmersdorf
39,Universität zu Köln,University of Cologne,Q54096,Köln
49,Goethe-Universität Frankfurt am Main,Goethe University Frankfurt,Q50662,Frankfurt am Main


In [16]:
# Kombiniere Labels mit Location
data_de = data[['label_de','item','location']]
data_de.columns=['label','item','location']
data_en = data[['label_en','item','location']]
data_en.columns=['label','item','location']
data = data_de.append(data_en)
data = data[~data['label'].isnull()]
data.head()

,label,item,location
0,Friedrich-Alexander-Universität Erlangen-Nürnberg,Q40025,Nürnberg
34,Martin-Luther-Universität Halle-Wittenberg,Q32120,Halle (Saale)
38,Technische Universität Berlin,Q51985,Charlottenburg-Wilmersdorf
39,Universität zu Köln,Q54096,Köln
49,Goethe-Universität Frankfurt am Main,Q50662,Frankfurt am Main


In [17]:
data[data['location'] == 'Erlangen']

,label,item,location


# Matching-Verfahren

Für das Matching wird die Cosinus-Similarity genutzt.

Dafür werden Strings in Trigramme vektorisiert, die dann von der Cosinus-Similarity verarbeitet werden.

Das Resultat ist ein Wert zwischen 0 (niedrige) und 1 (hohe Ähnlichkeit).

In [18]:
# transformiert string in n-grams
def ngrams(string, n=3):
    try:
        string = re.sub(r'[,-./]|\sBD',r'', string)
        ngrams = zip(*[string[i:] for i in range(n)])
        return [''.join(ngram) for ngram in ngrams]
    except TypeError:
        return None
    
# berechnet die Kosinus-Ähnlichkeit zwischen zwei Vektoren, siehe https://de.wikipedia.org/wiki/Kosinus-%C3%84hnlichkeit
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
# Funktion die jeden Input mit jedem Datenpunkt aus dem Wikidata-Datensatz abgleicht
def matching(org, city):
    label1 = Counter(ngrams(org))
    city1 = Counter(ngrams(city))

    simil=[]

    # iterierte hier durch wikidata
    for row in data.iterrows():
        label, city = row[1]['label'], row[1]['location']

        # zählt wie oft ein n-gram in einem Vektor vorkommt
        label2 = Counter(ngrams(label))
        city2 = Counter(ngrams(city))

        try:
            cosine_label = get_cosine(label1, label2)
        except TypeError:
            cosine_label = 0 

        try:
            cosine_city = get_cosine(city1, city2)
        except TypeError:
            cosine_city = 0 

        simil.append( (row[1]['item'], row[1]['label'], cosine_label, cosine_city) )

    results= pd.DataFrame(simil, columns=['qid','label','org_sim', 'city_sim'])

    return results.sort_values(['org_sim', 'city_sim'], ascending=[False, False])

In [19]:
# zunächst werden die jeweiligen Strings in Vektor-Trigramme umgeformt
t1 = ngrams('Universität', n=3)
t2 = ngrams('Universitaet', n=3)
print(t1)
print(t2)

# der Vorteil von Vektor-Trigrammen besteht darin, dass sie weniger anfällig gegenüber Schreibfehlern/-weisen sind:
# 8 von 9 Teilen von Universität sind auch in Universitaet

['Uni', 'niv', 'ive', 'ver', 'ers', 'rsi', 'sit', 'itä', 'tät']
['Uni', 'niv', 'ive', 'ver', 'ers', 'rsi', 'sit', 'ita', 'tae', 'aet']


In [20]:
# Die Kosinus-Similarity verwendet einen Bag-of-Words-Ansatz, interessierte sich also nur dafür wie oft ein n-gram vorkommt, aber nicht wo
Counter(t1), Counter(t2)

(Counter({'Uni': 1,
          'niv': 1,
          'ive': 1,
          'ver': 1,
          'ers': 1,
          'rsi': 1,
          'sit': 1,
          'itä': 1,
          'tät': 1}),
 Counter({'Uni': 1,
          'niv': 1,
          'ive': 1,
          'ver': 1,
          'ers': 1,
          'rsi': 1,
          'sit': 1,
          'ita': 1,
          'tae': 1,
          'aet': 1}))

In [21]:
# Die Kosinus-Ähnlichkeit zwischen Universität und Universitaet
get_cosine(Counter(t1), Counter(t2))

0.7378647873726218

### Matching

In [22]:
# iteriert durch Web of Science Datenpunkte und ruft die matching Funktion auf
# diese iteriert durch Wikidata Datenpunkte und berechnet die Cosinus-Similarity zwischen zwei Datenpunkten
for row in wos_sample[:10].iterrows():
    org, city = row[1]['ORGANIZATION1'], row[1]['CITY']
    
    result = matching(org=org, city=city)
    
    # Es wird nur der beste Wert ausgeben
    print(org, city , ' ---> ', result[:1].values[0])

Univ Konstanz Constance  --->  ['Q835440' 'Universität Konstanz' 0.6396021490668314 0.6172133998483676]
Deutsch Herzzentrum Munich Munich  --->  ['Q157808' 'Technical University Munich' 0.23570226039551584 0.0]
Friedrich Loeffler Inst Insel Riems  --->  ['Q17123243' 'Friedrichs-Polytechnikum' 0.3333333333333333 0.0]
Tech Univ Munich Am Coulombwall  --->  ['Q157808' 'Technical University Munich' 0.5657789498610036 0.0]
Univ Cologne Cologne  --->  ['Q54096' 'University of Cologne' 0.5803810000880093 0.0]
Univ Heidelberg Mannheim  --->  ['Q151510' 'Heidelberg University' 0.6362847629757777 0.0]
Univ Bayreuth Bayreuth  --->  ['Q702482' 'Universität Bayreuth' 0.6396021490668314 1.0000000000000002]
Tech Univ Dresden Dresden  --->  ['Q158158' 'Technische Universität Dresden' 0.5367450401216932
 0.9999999999999998]
Univ Leipzig Leipzig  --->  ['Q154804' 'Universität Leipzig' 0.6135719910778963 0.9999999999999998]
Univ Bremerhaven Bremerhaven  --->  ['Q1622093' 'Hochschule Bremerhaven' 0.597614

In [23]:
# One-Code-Window für Beispiele

word1 = "Constance"
word2 = "Konstanz"

get_cosine(Counter(ngrams(word1, n=3)), Counter(ngrams(word2, n=3)))

0.6172133998483676

### Die Herausfordung besteht nun darin eine Gewichtungsfunktion mit folgender Form zu finden:
### x1 * Similarity_ORGANIZATION1 + x2 * SIMILARITY_CITY > Threshold

Ein guter Threshold kann über Regeln, Machine Learning, Deep Learning etc. ermittelt werden.

#### Im Projekt haben wir die Cosinus-Matches gelabelt und dann mit den supervised Daten die besten Gewichte über Machine Learning ermittelt.

# Tweaked Matcher used in Project
only show case, not part of the workshop package

In [1]:
import pandas as pd, sqlite3

db = sqlite3.connect("/home/ckoss/Desktop/abd_workshop/abd_workshop_2021/data/sample.db")
cur = db.cursor()
df = pd.DataFrame(cur.execute("""select * from wos_institutions""").fetchall(), columns=[c[0] for c in cur.description])
wos_sample = df[df['COUNTRYCODE']=='DEU'][['PK_INSTITUTIONS','ORGANIZATION1','CITY']]

In [2]:
from abd.match.institutions.matcher import InstitutionMatcher as IM
matcher=IM(wos_db=False)

[17:25:56.960] Loaded dataset: wos_b_2020_matcher.db


In [3]:
for row in wos_sample[:20].iterrows():
    org, city = row[1]['ORGANIZATION1'], row[1]['CITY']
    
    if not city:
        city = ''
    
    try:
        result = matcher.find_wiki_id_by_name(organization1=org, city=city, countrycode='DEU')
        wiki = matcher.get_wiki_datapoint(result)

        print(org, city , ' ---> ', wiki.wiki_id, wiki.main_label)
    except AttributeError:
        print('Error unknown Institution:', org,city)

Univ Konstanz Constance  --->  Q835440 University of Konstanz
Deutsch Herzzentrum Munich Munich  --->  Q1205693 German Heart Center Munich
Friedrich Loeffler Inst Insel Riems  --->  Q1457808 Friedrich Loeffler Institute
Tech Univ Munich Am Coulombwall  --->  Q157808 Technical University of Munich
Univ Cologne Cologne  --->  Q54096 University of Cologne
Univ Heidelberg Mannheim  --->  Q151510 Heidelberg University
Univ Bayreuth Bayreuth  --->  Q702482 University of Bayreuth
Tech Univ Dresden Dresden  --->  Q158158 TU Dresden
Univ Leipzig Leipzig  --->  Q154804 Leipzig University
Univ Bremerhaven Bremerhaven  --->  Q1622093 University of Applied Sciences Bremerhaven
Tech Univ Bergakad Freiberg Freiberg  --->  Q689854 Freiberg University of Mining and Technology
Univ Tubingen Tubingen  --->  Q153978 University of Tübingen
Hahn Meitner Inst Berlin GmbH Berlin  --->  Q314578 Helmholtz-Zentrum Berlin
Univ Stuttgart Stuttgart  --->  Q122453 University of Stuttgart
Univ Gottingen Gottingen  --

In [4]:
# one test window

org = "Hauptgebäude der Technischen Universität Berlin"

result = matcher.find_wiki_id_by_name(organization1=org, countrycode='DEU')
wiki = matcher.get_wiki_datapoint(result)

print(org, ' ---> ', wiki.wiki_id, wiki.main_label)

Hauptgebäude der Technischen Universität Berlin  --->  Q51985 Technical University of Berlin


In [5]:
# Ein Datenpunkt in unserer Institutionendatenbank
wiki.main_label, wiki.main_label_processed, wiki.all_label[:10], wiki.location[:10], wiki.postalcode[:10]

('Technical University of Berlin',
 'technical university berlin',
 ('tub',
  'jm brlyn ltqny',
  'm hd brlyn lltknwlwjy',
  'technische hochschule charlottenburg',
  'tu berlin',
  'institouto tekhnologias tou berolinou',
  'tekhnologiko panepistemio tou berolinou',
  'technische universitat',
  'technische universitat berlin',
  'universidad tecnica berlin'),
 ('berlin',
  'peerlinnn',
  'brlin',
  'brlyn lmny',
  'sm lmny',
  'mdyn brlyn',
  'wly brlyn',
  'berlin deutschland',
  'bundeshauptstadt berlin',
  'be'),
 ('10775',
  '13491',
  '10400',
  '10406',
  '10855',
  '12307',
  '12529',
  '11829',
  '12541',
  '13363'))